<a href="https://colab.research.google.com/github/eyatab/Separate-the-ground-truth-OD-OC-from-the-refuge-dataset/blob/main/SEPERATE_GT_REFUGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#importation

In [2]:
from skimage.transform import resize
import cv2
import numpy as np
from PIL import Image
import imutils
import matplotlib.pyplot as plt
import os

##extract disc cup

In [3]:
def extract_number(filename):
    # Supprimer le "g" initial et convertir le reste en entier
    return int(filename[1:-4])

def extract_disc_and_cup_masks(disc_mask,cup_mask):
   #extract disc
        disc_mask[(disc_mask == 0).all(axis=2)] = [128, 128, 128]  # Gris pour disc
        disc_mask[(disc_mask == 128).all(axis=2)] = [0, 0, 0]
        disc_mask= 255 - disc_mask

        #extract cup
        cup_mask[(cup_mask == 128).all(axis=2)] = [255, 255, 255]  # Blanc pour cup
        cup_mask = 255 - cup_mask
        # Apply thresholding to create a binary mask disc
        disc_mask = cv2.cvtColor(disc_mask, cv2.COLOR_BGR2GRAY)
        #_, disc_mask = cv2.threshold(disc_mask, 128, 255, cv2.THRESH_BINARY)
        # Apply thresholding to create a binary mask cup
        cup_mask= cv2.cvtColor(cup_mask, cv2.COLOR_BGR2GRAY)
        _, cup_mask = cv2.threshold(cup_mask, 1, 255, cv2.THRESH_BINARY)

        # Taille cible finale
        target_size = (256, 256)

        # Effectuer une réduction progressive en plusieurs étapes
        while disc_mask.shape[0] > target_size[0] or disc_mask.shape[1] > target_size[1]:
              # Réduire l'image de moitié à chaque étape
              disc_mask = cv2.resize(disc_mask, (max(disc_mask.shape[1] // 2, target_size[1]), max(disc_mask.shape[0] // 2, target_size[0])))

        # Redimensionner l'image à la taille finale
              disc_mask= cv2.resize(disc_mask, target_size,cv2.INTER_AREA)

        # Effectuer une réduction progressive en plusieurs étapes
        while cup_mask.shape[0] > target_size[0] or cup_mask.shape[1] > target_size[1]:
        # Réduire l'image de moitié à chaque étape
              cup_mask = cv2.resize(cup_mask, (max(cup_mask.shape[1] // 2, target_size[1]), max(cup_mask.shape[0] // 2, target_size[0])))

        # Redimensionner l'image à la taille finale
              cup_mask= cv2.resize(cup_mask, target_size,cv2.INTER_AREA)

        #disc_mask = cv2.resize(disc_mask, (256, 256),interpolation=cv2.INTER_NEAREST)
        #cup_mask = cv2.resize(cup_mask, (256, 256),interpolation=cv2.INTER_NEAREST)

        return disc_mask, cup_mask

In [ ]:
def readimg(img):
  img = cv2.resize(img, (256, 256))
  return img

#Train data OD OC


In [ ]:
Path_trainGT='/content/drive/MyDrive/REFUGE/train/gts'
Path_trainIMG='/content/drive/MyDrive/REFUGE/train/Images'
image_files = os.listdir(Path_trainIMG) #nom files
# Trier les noms de fichiers en utilisant la fonction de tri personnalisée
image_files = sorted(image_files, key=extract_number)

# Initialize lists to store training data and labels
training_images = []
training_GTOD= []
training_GTOC= []
for image_file in image_files:
    image_path = os.path.join(Path_trainIMG, image_file)
    gt_file = os.path.splitext(image_file)[0] + '.bmp'
    gt_path = os.path.join(Path_trainGT, gt_file)

    if os.path.isfile(gt_path):
        # Charger l'image et ground truth
        image = cv2.imread(image_path)
        gt = cv2.imread(gt_path)
        #create copy
        disc_mask = gt.copy()
        cup_mask = gt.copy()
        image= readimg(image)

        disc_mask, cup_mask = extract_disc_and_cup_masks(disc_mask, cup_mask)
        # Remodeler training_GTOD/ OC en (256, 256, 1)
        disc_mask= np.reshape(disc_mask, (256, 256, 1))
        cup_mask= np.reshape(cup_mask, (256, 256, 1))
        # Ajouter les données chargées aux listes
        training_images.append(image)
        training_GTOD.append(disc_mask)
        training_GTOC.append(cup_mask)

# Convertir les listes en tableaux NumPy pour une manipulation plus facile
training_images= np.array(training_images)
training_GTOD = np.array(training_GTOD)
training_GTOC = np.array(training_GTOC)

#Normalization

In [ ]:
training_images= training_images / 255.0
training_GTOD= training_GTOD / 255.0